# Туториал по использованию предобученной модели RuBERT в задаче классификации твитов по тональности

In [3]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers

In [4]:
# загружаем 5000 позитивных и негативных твитов

df_tweets = pd.read_csv('corpus.csv')
df_tweets = df_tweets.dropna()

In [5]:
df_tweets.head(2)

,Unnamed: 0,Unnamed: 0.1,text,id,channel,views_count_2,popularity,url,target
0,1,2738,https telegra ph file fac41ad20abc38747e8a8 jp...,551,datarootlabs,3600,1.241379,https://t.me/datarootlabs/551,1
1,3,2241,анализ выживаемости survival analysis класс ст...,421,datalytx,2300,1.352941,https://t.me/datalytx/421,1


In [6]:
df_tweets.shape

(8012, 9)

In [7]:
# создаем токенайзер для модели BERT, для его инициализации достаточно указать словарь, на котором обучалась предобученная модель
# BERT использует собственную токенизацию, никакой предобработки 

tokenizer = ppb.BertTokenizer(vocab_file='vocab.txt')

In [8]:
# токенизируем текст каждого твита, для BERT не требуется никакая дополнительная предобработка, лемматизация и прочее

tokenized = df_tweets['text'].astype(str).str[:512].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)) )

In [9]:
df_tweets['text'].astype(str).str[:512]

0       https telegra ph file fac41ad20abc38747e8a8 jp...
1       анализ выживаемости survival analysis класс ст...
2       одной из первых моих задач тут было внести нов...
3       яндекс вчера выпустил тепловую карту цен на не...
4       примерно так мы представляем себе студентов на...
                              ...                        
8007    15 часов видео про ml https www r bloggers com...
8008    i have presented quantitative analysis of text...
8009    https telegra ph file f8822e71b343d996bb29b pn...
8010    о важности подбора правильной визуализации htt...
8011    я думаю все из вас знают shopify конструктор д...
Name: text, Length: 8012, dtype: object

In [10]:
# Пример токенизации текста: на входе - текст, а на выходе имеем массив с номерами токенов из словаря модели BERT

print(df_tweets['text'][0])
print(tokenized[0])
print(tokenizer.tokenize(df_tweets['text'][0]))

https telegra ph file fac41ad20abc38747e8a8 jpg openai musenet для генерации музыки musenet нейронная сеть которая генерирует музыку из первых 5 нот для этого она использует множество различных инструментов и стилей послушать на soundcloud https soundcloud com openai audio статья https openai com blog musenet
[101, 74034, 65041, 14751, 233, 23050, 96878, 49404, 36756, 2190, 2748, 3599, 237, 14962, 12551, 151, 241, 153, 233, 153, 45141, 14599, 12054, 82948, 12573, 271, 949, 58628, 13772, 82948, 12573, 271, 15175, 1997, 960, 10349, 1901, 72764, 11559, 883, 2980, 146, 29734, 949, 1250, 1151, 13604, 6979, 9666, 23971, 322, 5291, 323, 17635, 801, 19814, 88256, 74034, 19814, 88256, 6502, 14599, 12054, 77325, 7467, 7413, 74034, 14599, 12054, 6502, 98988, 82948, 12573, 271, 102]
['https', 'tele', '##gr', '##a', 'ph', 'file', 'fac', '##41', '##ad', '##20', '##ab', '##c', '##38', '##74', '##7', '##e', '##8', '##a', '##8', 'jpg', 'open', '##ai', 'mus', '##ene', '##t', 'для', 'генерации', 'музыки'

In [11]:
# инициализируем предобученную модель RuBERT из файла, 
# в json-файле конфигурации описаны параметры модели

config = ppb.BertConfig.from_json_file('bert_config.json')
model = ppb.BertModel.from_pretrained('pytorch_model.bin', config = config)

In [12]:
model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [13]:
# из-за того, что каждый твит в датасете имеет разную длину (количество токенов)
# мы делаем паддинг - заполнение нулями каждого массива токенов до длины максимального массива
# чтобы на выходе получить матрицу из токенизированных текстов одной длины

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [14]:
# посмотрим на размерность матрицы токенизированных твитов после паддинга

np.array(padded).shape

(8012, 310)

In [15]:
# Накладываем маску на значимые токены
# В данном случае нам важны все слова кроме нулевых токенов, появившихся на предыдущем шаге паддинга

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(8012, 310)

In [16]:
# а теперь сформируем вектора текстов с помощью модели RuBERT

# это не быстрый процесс, импортируем инструмент для визуализации времени обработки в цикле
from tqdm import tqdm

# для того, чтобы модель отработала в условиях ограниченных ресурсов - оперативной памяти, мы разделяем входной датасет на батчи.
# при батче в 100 твитов потребление оперативной памяти укладывается в 1Гб
batch_size = 100

# Делаем пустой список для хранения эмбеддингов (векторов) твитов
embeddings = []

for i in tqdm(range(padded.shape[0] // batch_size)):
        # преобразуем батч с токенизированными твитами в тензор 
        # по сути тензор - это многомерный массив, который может быть обработан нейронной сетью
        input_ids = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to('cuda') 
        
        # создаем тензор и для подготовленной маски
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).to('cuda')
        
        # передаем в модель BERT тензор из твитов и маску - на выходе получаем эмбеддинги - вектор текста твита
        # torch.no_grad() - для ускорения инференса модели отключим рассчет градиентов
        with torch.no_grad():
               last_hidden_states = model(input_ids, attention_mask=attention_mask_batch)
        
        # в итоге собираем все эмбеддинги твитов в features
        embeddings.append(last_hidden_states[0][:,0,:].cpu().numpy())



100%|██████████| 80/80 [00:54<00:00,  1.46it/s]


In [17]:
# преобразуем список батчей эмбеддингов в numpy-матрицу 
features = np.concatenate(embeddings)

In [18]:
# выводим размерность полученной матрицы эмбеддингов
# данная модель BERT формирует вектора текстов в 768-мерном пространстве признаков
features.shape

(8000, 768)

In [19]:
# Импортируем необходимые библиотеки для обучения классификатора на logreg и оценки качества

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import os
from sklearn import preprocessing
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split,GridSearchCV
import matplotlib.pyplot as plt
import scikitplot as skplt
import seaborn as sns

In [20]:
# Сохраним целевую переменную: метку тональности позитив/негатив

labels = df_tweets['target']

In [21]:
labels

0       1
1       1
2       1
3       1
4       1
       ..
8007    1
8008    0
8009    0
8010    1
8011    1
Name: target, Length: 8012, dtype: int64

In [22]:
features.shape

(8000, 768)

In [23]:
labels.shape

(8012,)

In [24]:
# Разделяем матрицу признаков и целевую переменную на обучающий и тестовый набор

train_features, test_features, train_labels, test_labels = train_test_split(features, labels[:8000], test_size=0.1, random_state=42)

In [25]:
catboost_params = {
    'iterations': 10000,
    'learning_rate': 0.001,
    'eval_metric': 'AUC',
    #'eval_metric': 'Accuracy',
    'loss_function': 'MultiClass',
    'task_type': 'GPU',
    'early_stopping_rounds': 1000,
    'use_best_model': True,
    'objective':"MultiClass",
    'verbose': 100
}

In [26]:
X_tr = pd.DataFrame(train_features) #X_train
X_val =  pd.DataFrame(test_features)
y_tr = train_labels
y_val = test_labels

train_pool = Pool(
    X_tr, 
    y_tr
)
valid_pool = Pool(
    X_val, 
    y_val
)

In [27]:
model = CatBoostClassifier(**catboost_params)
model.fit(train_pool, eval_set=valid_pool)

AUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.5825471	best: 0.5825471 (0)	total: 11.1ms	remaining: 1m 50s
100:	test: 0.6461223	best: 0.6477594 (94)	total: 996ms	remaining: 1m 37s
200:	test: 0.6538916	best: 0.6541755 (199)	total: 1.95s	remaining: 1m 35s
300:	test: 0.6584509	best: 0.6586210 (287)	total: 2.88s	remaining: 1m 32s
400:	test: 0.6600336	best: 0.6608843 (368)	total: 3.76s	remaining: 1m 30s
500:	test: 0.6631843	best: 0.6634453 (494)	total: 4.63s	remaining: 1m 27s
600:	test: 0.6659155	best: 0.6659155 (600)	total: 5.5s	remaining: 1m 26s
700:	test: 0.6673158	best: 0.6674136 (699)	total: 6.46s	remaining: 1m 25s
800:	test: 0.6672783	best: 0.6674808 (764)	total: 7.38s	remaining: 1m 24s
900:	test: 0.6688452	best: 0.6694066 (893)	total: 8.28s	remaining: 1m 23s
1000:	test: 0.6716665	best: 0.6719069 (997)	total: 9.18s	remaining: 1m 22s
1100:	test: 0.6738043	best: 0.6738043 (1100)	total: 10.1s	remaining: 1m 21s
1200:	test: 0.6745752	best: 0.6747139 (1154)	total: 11s	remaining: 1m 20s
1300:	test: 0.6754889	best: 0.6754889 (1

In [28]:
import random

In [35]:
# делаем пробное предсказание
tweet_index = random.randint(1,8000)

print('Text: ' + df_tweets['text'][tweet_index])
print('Predict label: ', model.predict(features[tweet_index:tweet_index+1][:])[0])
print('True label: ', df_tweets['target'][tweet_index])


Text: why are so many people paying so much and putting themselves at risk for a placebo effect via the upshot http ift tt 2spavjc
Predict label:  [0]
True label:  0


In [36]:
# оцениваем accuracy на тестовой выборке

model.score(test_features, test_labels)

0.60375

In [45]:
d = []
for idx, tt in enumerate(tqdm(df_tweets['text'].head(4000))):
    try:
        d.append(
            {
                'text': tt,
                'predict': model.predict(features[idx:idx+1][:])[0][0],
                'gtrue': df_tweets['target'][idx]
            }
        )
    except:
        pass
    #print(max(res,key=itemgetter(1))[0])    
df_train = pd.DataFrame(d)    

100%|██████████| 4000/4000 [00:15<00:00, 265.43it/s]


In [46]:
df_train.predict.value_counts()

0    3650
1     261
2      89
Name: predict, dtype: int64

In [47]:
df_train.gtrue.value_counts()

0    2410
1    1032
2     558
Name: gtrue, dtype: int64

In [48]:
import pandas as pd

def confusion_matrix(df: pd.DataFrame, col1: str, col2: str):
    """
    Given a dataframe with at least
    two categorical columns, create a 
    confusion matrix of the count of the columns
    cross-counts
    
    use like:
    
    >>> confusion_matrix(test_df, 'actual_label', 'predicted_label')
    """
    return (
            df
            .groupby([col1, col2])
            .size()
            .unstack(fill_value=0)
            )

In [49]:
confusion_matrix(df_train,'predict','gtrue')

gtrue,0,1,2
predict,,,
0,2393,790,467
1,13,237,11
2,4,5,80


In [62]:
# обучаем классификатор на основе логистической регрессии

lr_clf = LogisticRegression(random_state=42, class_weight='balanced')
lr_clf.fit(train_features, train_labels)

/home/renat/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(class_weight='balanced', random_state=42)

In [63]:
import random

In [64]:
# делаем пробное предсказание
tweet_index = random.randint(1,8000)

print('Text: ' + df_tweets['text'][tweet_index])
print('Predict label: ', lr_clf.predict(features[tweet_index:tweet_index+1][:])[0])
print('True label: ', df_tweets['target'][tweet_index])


Text: migration racial or ethnic self identity and marriage were among the many topics explored at the population association of america s annual meeting last month via pew research center https ift tt 2imome2
Predict label:  0
True label:  0


In [65]:
# оцениваем accuracy на тестовой выборке

lr_clf.score(test_features, test_labels)

0.5125

In [66]:
d = []
for idx, tt in enumerate(tqdm(df_tweets['text'].head(4000))):
    try:
        d.append(
            {
                'text': tt,
                'predict': lr_clf.predict(features[idx:idx+1][:])[0],
                'gtrue': df_tweets['target'][idx]
            }
        )
    except:
        pass
    #print(max(res,key=itemgetter(1))[0])    
df_train = pd.DataFrame(d)    
    

100%|██████████| 4000/4000 [00:00<00:00, 13291.46it/s]


In [67]:
df_train.predict.value_counts()

0    1867
1    1210
2     923
Name: predict, dtype: int64

In [68]:
df_train.gtrue.value_counts()

0    2410
1    1032
2     558
Name: gtrue, dtype: int64

In [69]:
import pandas as pd

def confusion_matrix(df: pd.DataFrame, col1: str, col2: str):
    """
    Given a dataframe with at least
    two categorical columns, create a 
    confusion matrix of the count of the columns
    cross-counts
    
    use like:
    
    >>> confusion_matrix(test_df, 'actual_label', 'predicted_label')
    """
    return (
            df
            .groupby([col1, col2])
            .size()
            .unstack(fill_value=0)
            )

In [70]:
confusion_matrix(df_train,'predict','gtrue')

gtrue,0,1,2
predict,,,
0,1479,275,113
1,532,609,69
2,399,148,376
